## Exemplary SA-CCR allocation under consideration of an initial margin threshold

Our goal is to perform an Euler allocation for the minimal example of a one trade portfolio.
We use the same 200Bn IRS as in previous examples.

In [1]:
import QuantLib as ql
from IPython.core.display import display, Markdown
from scipy import optimize
from allocation.eulerAllocator import EulerAllocator
from collateralAgreement.collateralAgreement import CollateralAgreement
from instruments.equity_instruments.equityOption import EquityOption
from instruments.interestRateInstrument.irs import IRS
from jupyterUtils import export
from marketdata.interestRateIndices import InterestRateIndex
from marketdata import init_marketdata
from sa_ccr.sa_ccr import SA_CCR
from utilities.Enums import Stock, TradeType, TradeDirection
asdf =1

In [2]:
irs = IRS(notional=200000000000,
          index=InterestRateIndex.USDLIBOR3M,
          timeToSwapStart=ql.Period(2, ql.Days),
          timeToSwapEnd=ql.Period(10, ql.Years))

In [3]:
ca = CollateralAgreement(threshold=2000000000,
                         mta=0)
ca.link_sa_ccr_instance(SA_CCR(ca))
ca.add_trades(irs)

The inclusion of the threshold raises the ead since it lowers the available overcollataralization.

In [4]:
ead_with_threshold = ca.get_sa_ccr_model().get_risk_measure()
print(ead_with_threshold)
ca.threshold = 0
ead_no_threshold = ca.get_sa_ccr_model().get_risk_measure()
print(ead_no_threshold)
ca.threshold = 2000000000

817798882.3982577
582881953.4648591


In [5]:
display(Markdown('The EAD with threshold is %.2f while the EAD without threshold is %.2f' %(ead_with_threshold, ead_no_threshold)))

The EAD with threshold is 817798882.40 while the EAD without threshold is 582881953.46

When trying to allocate with threshold we realize that the allocation is not working due to the missing homogeneity of C.

In [6]:
eulerAllocator = EulerAllocator(ca)
allocated_value = eulerAllocator.allocate_ead()[irs]
print(allocated_value)

526604164.2713547


In [7]:
display(Markdown('The value of %.2f that has been allocated to the single trade in the portfolio is far off from the portfolios EAD of %.2f' % (allocated_value, ead_with_threshold)))

The value of 526604164.27 that has been allocated to the single trade in the portfolio is far off from the portfolios EAD of 817798882.40

If we instead allocate without threshold, the allocation works.

In [8]:
ca.threshold = 0
allocated_value=eulerAllocator.allocate_ead()[irs]
print(allocated_value)

582887839.6499157


In [9]:
display(Markdown('The value of %.2f that has been allocated to the single trade in the portfolio equals the portfolios EAD without threshold of %.2f' % (allocated_value, ead_no_threshold)))

The value of 582887839.65 that has been allocated to the single trade in the portfolio equals the portfolios EAD without threshold of 582881953.46

A reasonable approach to allocate an SA-CCR EAD under consideration of a threshold could be to allocate without threshold and then scale accordingly:

\begin{align}
\label{eq:C threshold scaling}
    X_{t\text{, TH}} = X_{t\text{, no TH}} \frac{EAD_{\text{TH}}}{EAD_{\text{no TH}}}
\end{align}

In [10]:
print(allocated_value * (ead_with_threshold/ead_no_threshold))


817807140.8724346


However, this approach does coincide with a loss of precision.
If the $C_{calc}$ is below the threshold then $C$ is 0 and exhibits homogeneity, even when taking the threshold into account.
We can construct an example that shows, that the approach in \ref{eq:C threshold scaling} does not yield the correct allocation which can be calculated when taking the threshold into account since the IM is below the IM threshold.

We set up an IRS and an Equity option that have a similar EAD on their own. Here we calculate with a threshold of 50Mn which is a common value as it is the maximum amount permitted by the regulator. \todo{Cite source for this}

In [11]:
irs = IRS(notional=100000000,
          index = InterestRateIndex.USDLIBOR3M,
          timeToSwapStart=ql.Period(2, ql.Days),
          timeToSwapEnd=ql.Period(10, ql.Years))

ca = CollateralAgreement(threshold=50000000)
ca.link_sa_ccr_instance(SA_CCR(ca))
ca.add_trades(irs)

In [12]:
print(ca.get_im_model().get_risk_measure())
print(ca.get_sa_ccr_model().get_risk_measure())

4519078.535740282
1651404.7245623078


In [13]:
def find_notional_for_eq_ead(notional):
   eqOpt = EquityOption(notional = notional)
   global ca
   ref_ead = ca.get_sa_ccr_model().get_risk_measure()
   trades = ca.trades
   ca.remove_all_trades()
   ca.add_trades(eqOpt)
   compare_ead = ca.get_sa_ccr_model().get_risk_measure()
   ca.remove_all_trades()
   ca.add_trades(trades)
   return (ref_ead - compare_ead)**2

eqOptNot = optimize.minimize(find_notional_for_eq_ead, x0=10000)

In [28]:
display(Markdown('Through optimization it can be identified that for a one year long call option on Adidas an underlying stock count of %d stocks results in the same EAD as for the IRS above.' %eqOptNot.x[0]))

Through optimization it can be identified that for a one year long call option on Adidas an underlying stock count of 403106 stocks results in the same EAD as for the IRS above.

In [15]:
eqOpt = EquityOption(notional = 403106,
                     maturity = ql.Period(1, ql.Years),
                     underlying = Stock.ADS,
                     tradeType = TradeType.CALL,
                     tradeDirection = TradeDirection.LONG)

In [16]:
eqOpt = EquityOption(notional = eqOptNot.x[0])
ca2 = CollateralAgreement(threshold = 50000000)
ca2.link_sa_ccr_instance(SA_CCR(ca2))
ca2.add_trades(eqOpt)

The initial margin of this single trade is also far below the 50Mn threshold but differs significantly from the IM of the IRS.

In [32]:
print('EAD: ' + str(ca2.get_sa_ccr_model().get_risk_measure()))
print('IM:  ' + str(ca2.get_im_model().get_risk_measure()))

EAD: 1651404.7245622796
IM:  2802685.461885011


In [17]:
display(Markdown('Given the market data, difference in model, risk horizon etc. the SA CCR EAD model calculates the same risk for the two trades when calculated individually, while the ISDA SIMM IM model evaluates the IRS to be %d%% riskier.' %((ca.get_im_model().get_risk_measure()/ca2.get_im_model().get_risk_measure()-1)*100)))

Given the market data, difference in model, risk horizon etc. the SA CCR EAD model calculates the same risk for the two trades when calculated individually, while the ISDA SIMM IM model evaluates the IRS to be 61% riskier.

When putting both trades in a common portfolio we observe, that the EAD and the IM of this joint portfolio is the sum of the two separate portfolios.
This is not surprising since both, the SA CCR and ISDA SIMM model do not recognize any hedge effect between different asset classes.

In [18]:
ca3 = CollateralAgreement(threshold=50000000)
ca3.link_sa_ccr_instance(SA_CCR(ca3))
ca3.add_trades([irs, eqOpt])

In [19]:
display(Markdown(
    'The EAD of the combined portfolio is %d USD.' %ca3.get_sa_ccr_model().get_risk_measure() +\
    ' The calculated IM of the combined portfolio is %d USD.' %ca3.get_im_model().get_risk_measure()
))

The EAD of the combined portfolio is 3302809 USD. The calculated IM of the combined portfolio is 7321763 USD.

In [20]:
eulerAllocator = EulerAllocator(ca3)
ca3.threshold = 50000000
allocation = eulerAllocator.allocate_ead()

In [21]:
display(Markdown(
    'Euler allocation allocates %d USD of the EAD to the IRS.' %allocation[irs] +\
    'Euler allocation allocates %d USD of the EAD to the equity option.' %allocation[eqOpt] +\
    'The sum of these two allocations is %d USD, ' %sum([alloc for alloc in allocation.values()]) +\
    'Which is close to the EAD calculated for the portfolio of %d USD.' %ca3.get_sa_ccr_model().get_risk_measure()
))

Euler allocation allocates 1651404 USD of the EAD to the IRS.Euler allocation allocates 1651404 USD of the EAD to the equity option.The sum of these two allocations is 3302809 USD, Which is close to the EAD calculated for the portfolio of 3302809 USD.

Due to the high threshold, no IM is exchanged. Only VM is exchanged which is not overcollateralization and therefore only reduces the $RC$ in formula \todo{reference RC+PFE formula} to 0 but does not impact the $PFE$.

In [22]:
display(Markdown('The available collateral is %d USD. ' %ca3.get_C() + 'Due to the hight threshold this consists only of the VM which is %d USD.' %ca3.get_vm_model().get_vm()))

The available collateral is 1450296 USD. Due to the hight threshold this consists only of the VM which is 1450296 USD.

The 50/50 allocation is certainly the correct result since there are no hedge effects between the two trades and they both have the same stand alone EAD.
However, when applying formula \ref{eq:C threshold scaling} we yield a different result.

In [23]:
ead_with_threshold = ca3.get_sa_ccr_model().get_risk_measure()
ca3.threshold=0
ead_no_threshold = ca3.get_sa_ccr_model().get_risk_measure()
allocation = eulerAllocator.allocate_ead()
factor = ead_with_threshold/ead_no_threshold
print(allocation[irs]*factor)
ca3.threshold = 50000000

1153317.0534241588


In [24]:
display(Markdown('Euler allocation allocates %d USD of the EAD to the IRS.' %(allocation[irs]*factor) + \
                 ' Euler allocation allocates %d USD of the EAD to the equity option.' %(allocation[eqOpt]*factor) + \
                 ' The sum of these two allocations is %d USD, ' %(sum([alloc for alloc in allocation.values()])*factor) + \
                 'which is close to the EAD calculated for the portfolio of %d USD.' %ca3.get_sa_ccr_model().get_risk_measure()))

Euler allocation allocates 1153317 USD of the EAD to the IRS. Euler allocation allocates 2149501 USD of the EAD to the equity option. The sum of these two allocations is 3302819 USD, which is close to the EAD calculated for the portfolio of 3302809 USD.

Therefore, the approximation of \ref{eq:C threshold scaling} should only be used if the $IM_{calc} > TH_{IM}$.


In [33]:
export('Exemplary SA-CCR allocation considering an initial margin threshold.ipynb')